# Nested Models & Robust Error Handling

This notebook demonstrates how to work with **complex nested Pydantic models** and implement **robust error handling** using Instructor. 

This cookbook shows how to:
- Use [complex nested Pydantic models](https://python.useinstructor.com/getting-started/#working-with-complex-models) (enums, regex, conditional fields)
- Apply [validation rules and automatic retry/repair patterns](https://python.useinstructor.com/getting-started/#validation-and-error-handling)
- Perform [streaming extraction](https://python.useinstructor.com/getting-started/#streaming-responses) (async generator style)

## Prerequisites
Before getting started, you'll need:

- A [Writer AI Studio](https://app.writer.com/register) account
- An API key, which you can obtain by following the [API Quickstart](https://dev.writer.com/home/quickstart)



## Install dependencies

Run this cell once in the notebook environment to install the needed libraries.

In [ ]:
%pip install "instructor[writer]" writer-sdk python-dotenv pydantic

## Environment setup

Set the `WRITER_API_KEY` environment variable. We recommend setting it in a `.env` file in the root of your project, but this tutorial will set it in an environment variable if you don't have a `.env` file.

In [ ]:
import getpass
import os

import instructor
from writerai import Writer


if not os.getenv("WRITER_API_KEY"):
    os.environ["WRITER_API_KEY"] = getpass.getpass("Enter your Writer API key: ")

client = instructor.from_writer(Writer(api_key=os.getenv('WRITER_API_KEY')))

## Imports and clients(Complex Models)

This section imports required libraries, defines Enum-based role types, builds nested Pydantic models with validation, and prepares the Instructor-powered Writer AI client for structured output.

In [ ]:
from typing import List, Optional, Literal, Union, Iterable, Type
from pydantic import BaseModel, Field, field_validator
from enum import Enum
import re
import asyncio

class Role(str, Enum):
    employee = 'employee'
    contractor = 'contractor'
    intern = 'intern'

class Address(BaseModel):
    street: str
    city: str
    state: str
    postal_code: str = Field(..., description='ZIP or ZIP+4')
    country: Optional[str] = 'USA'

    @field_validator('postal_code')
    def zip_must_match(cls, v):
        if not re.match(r'^\d{5}(-\d{4})?$', v):
            raise ValueError('postal_code must be 5 digits or ZIP+4')
        return v

class Person(BaseModel):
    full_name: str
    age: int = Field(..., gt=0, lt=120)
    role: Role
    email: Optional[str] = None
    addresses: List[Address] = Field(default_factory=list)

    @field_validator('full_name')
    def must_have_first_and_last(cls, v):
        if len(v.split()) < 2:
            raise ValueError('full_name must include first and last name')
        return v

### 📌 Example: Extracting a Person Object from Unstructured Text

 - The input text is informal and unstructured. Instructor parses it, extracts relevant fields, and constructs a `Person` object. Pydantic automatically validates full name format, age range, role enum, email, and ZIP code format.


In [60]:
example_messages_person = [
    {"role": "user", "content": "John Smith, 35, employee, john.smith@acme.com, address: 123 Main St, Springfield, IL 62704"},
]

user = client.create(
    model="palmyra-x5",
    messages=example_messages_person,
    response_model=Person,
)
print(user)

full_name='John Smith' age=35 role=<Role.employee: 'employee'> email='john.smith@acme.com' addresses=[Address(street='123 Main St', city='Springfield', state='IL', postal_code='62704', country='USA')]


### 📌 Example: Extracting a Person Object from Detailed Narrative

 - Even though the input contains multiple sentences and extra context about John’s work and activities, Instructor parses the narrative, identifies the relevant details, and constructs a `Person` object. Pydantic ensures that fields such as full name, age, role, email, and postal code are correctly validated.


In [61]:
example_messages_person = [
    {"role": "user", "content": (
"We were working with John Smith, a 35-year-old employee at our company. John has been involved in several key projects over the past year, contributing both to team planning and execution. He is known for his attention to detail, reliability, and collaborative approach."
"He can be reached via email at john.smith@acme.com,"
"and his office is located at 123 Main St, Springfield, IL 62704. In addition to his regular duties, John occasionally mentors new team members and participates in cross-department initiatives, which has helped improve workflow efficiency."
    )},
]

user = client.create(
    model="palmyra-x5",
    messages=example_messages_person,
    response_model=Person,
)

print(user)

full_name='John Smith' age=35 role=<Role.employee: 'employee'> email='john.smith@acme.com' addresses=[Address(street='123 Main St', city='Springfield', state='IL', postal_code='62704', country='USA')]


### 📌 Example: Handling Partial or Ambiguous Input with Retries

 - By using `max_retries`, the request can be automatically retried to attempt better parsing or clarification. If the model still fails to generate a valid response, the exception handling captures both the last attempted message content and the underlying exception, allowing developers to debug or provide additional guidance.


In [62]:
example_messages_person = [
    {"role": "user", "content": "John"},
]
try:
    user = client.create(
        model="palmyra-x5",
        messages=example_messages_person,
        response_model=Person,
        max_retries=2
    )
except Exception as e:
    print(f"content: {e.failed_attempts[0].completion.choices[0].message.content}")
    print(f"exception: {e.failed_attempts[0].exception}")

content: To provide assistance, I need more information about John. Is there something specific you'd like to know or discuss about John, or perhaps you need help with a particular task related to someone with that name?
exception: Instructor does not support multiple tool calls, use List[Model] instead


## Streaming extraction example

This example demonstrates how to perform **streaming extraction** using Instructor.

This approach is useful for:
- Monitoring structured extraction in real time
- Handling very long or complex inputs without waiting for the full response
- Building pipelines that can act on partially extracted data immediately

Each chunk represents a progressively completed `Person` object, with fields like name, age, role, email, and addresses gradually populated as the model processes the input.


In [63]:
example_messages_person = [
    {"role": "user", "content": (
"We were working with John Smith, a 35-year-old employee at our company. John has been involved in several key projects over the past year, contributing both to team planning and execution. He is known for his attention to detail, reliability, and collaborative approach."
"He can be reached via email at john.smith@acme.com,"
"and his office is located at 123 Main St, Springfield, IL 62704. In addition to his regular duties, John occasionally mentors new team members and participates in cross-department initiatives, which has helped improve workflow efficiency."
    )},
]

stream = client.chat.completions.create_partial(
    model="palmyra-x5",
    messages=example_messages_person,
    response_model=Person,
)
for chunk in stream:
    print(chunk)

full_name='John Smith' age=None role=None email=None addresses=None
full_name='John Smith' age=35 role=None email=None addresses=None
full_name='John Smith' age=35 role=<Role.employee: 'employee'> email=None addresses=None
full_name='John Smith' age=35 role=<Role.employee: 'employee'> email='john.smith@acme.com' addresses=None
full_name='John Smith' age=35 role=<Role.employee: 'employee'> email='john.smith@acme.com' addresses=[PartialAddress(street='123 Main St', city='Springfield', state='IL', postal_code='62704', country=None)]
